# Case 1

## Packages needed

In [ ]:
import scipy.io
import numpy as np
import pandas as pd
from sklearn import preprocessing as preproc # load preprocessing function
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
from sklearn.model_selection import KFold, train_test_split
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer
import warnings # to silence convergence warnings

# seaborn can be used to "prettify" default matplotlib plots by importing and setting as default
import seaborn as sns
sns.set() # Set searborn as default

#being used to check dir and change of dir
import os

## Load Training data

In [ ]:
#used to load all missing values to the same format
missing_values = ["n.a.","NA","n/a", "na", " NaN","NaN"]
df = pd.read_csv('case1Data.txt', delimiter = "," ,na_values=missing_values, skipinitialspace=True)

In [ ]:
#remove space in heading
df.columns = df.columns.str.replace('_ ','_').str.strip()

In [ ]:
#splitting response and predictors from each other.
y, X = df['y'], df.drop(['y'],axis=1)

In [ ]:
X

## Load Prediction data

In [ ]:
#used to load all missing values to the same format
missing_values = ["n.a.","NA","n/a", "na", " NaN","NaN"]
df_pred = pd.read_csv('case1Data_Xnew.txt', delimiter = ",",na_values=missing_values, skipinitialspace=True)

In [ ]:
#remove space in heading
df_pred.columns = df_pred.columns.str.replace('_ ','_').str.strip()

In [ ]:
df_pred

## Data Prep

### Missing values - Training Data
There are multiple ways to handle missing values. But the simplest one is simply to one-hot-encode the categorical variables, where NaN also will be a variable. 

https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4

In [ ]:
#Start by locating where the missing values are
columns_with_missing_values = X.columns[X.isnull().any()]
X[columns_with_missing_values].isnull().sum()

In [ ]:
# To hold variable names
labels = [] 

# To hold the count of missing values for each variable 
valuecount = [] 

# To hold the percentage of missing values for each variable
percentcount = [] 

for col in columns_with_missing_values:
    labels.append(col)
    valuecount.append(X[col].isnull().sum())
    # X.shape[0] will give the total row count
    percentcount.append(X[col].isnull().sum()/X.shape[0])

ind = np.arange(len(labels))

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,18))

rects = ax1.barh(ind, np.array(valuecount), color='blue')
ax1.set_yticks(ind)
ax1.set_yticklabels(labels, rotation='horizontal')
ax1.set_xlabel("Count of missing values")
ax1.set_title("Variables with missing values")

rects = ax2.barh(ind, np.array(percentcount), color='pink')
ax2.set_yticks(ind)
ax2.set_yticklabels(labels, rotation='horizontal')
ax2.set_xlabel("Percentage of missing values")
ax2.set_title("Variables with missing values");

Above illustrates the counts and the percentage of each of the predictors, where there are missing values. For both C1 and C2 there are 15 missing values. 

In [ ]:
# Lets import seaborn. We will use seaborn to generate our charts
import seaborn as sns

# We will import matplotlib to resize our plot figure
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(20, 10))

# cubehelix palette is a part of seaborn that produces a colormap
cmap = sns.cubehelix_palette(light=1, as_cmap=True, reverse=True)
sns.heatmap(df.isnull(), cmap=cmap);

From the analysis above, we can see that we're only missing values in the categorical variables. Hence why, use subsection 9.2.4 in the book. This subsection describes how to treat missing values for categorical variables for three-based methods.


It states the following : "The first is applicable to categorical predictors: we simply make a new category for “missing.” From this we might discover that observations with missing values for some measurement behave differently than those with nonmissing values."

Reasons why we don't one-hot: https://towardsdatascience.com/one-hot-encoding-is-making-your-tree-based-ensembles-worse-heres-why-d64b282b5769

In [ ]:
X = X.fillna(value="Missing")

In [ ]:
labelencoder = preproc.LabelEncoder()

In [ ]:
# Assigning numerical values and storing in another column
X['C_1'] = labelencoder.fit_transform(X['C_1'])
X['C_2'] = labelencoder.fit_transform(X['C_2'])
X['C_3'] = labelencoder.fit_transform(X['C_3'])
X['C_4'] = labelencoder.fit_transform(X['C_4'])
X['C_5'] = labelencoder.fit_transform(X['C_5'])

The same is done for the other dataset, that is used for predictions.

In [ ]:
#Start by locating where the missing values are
columns_with_missing_values = df_pred.columns[df_pred.isnull().any()]
df_pred[columns_with_missing_values].isnull().sum()

In [ ]:
df_pred = df_pred.fillna(value="Missing")

In [ ]:
# Assigning numerical values and storing in another column
df_pred['C_1'] = labelencoder.fit_transform(df_pred['C_1'])
df_pred['C_2'] = labelencoder.fit_transform(df_pred['C_2'])
df_pred['C_3'] = labelencoder.fit_transform(df_pred['C_3'])
df_pred['C_4'] = labelencoder.fit_transform(df_pred['C_4'])
df_pred['C_5'] = labelencoder.fit_transform(df_pred['C_5'])

## Exploratory analysis

This is done so we have an understanding of the dataset. This can be done in multiple steps as

- Data Cleaning
- Multivariate analysis

Some of the data cleaning is done above, but lets investigate if further data cleaning is necessary. 

In [ ]:
#Multivariate outliers
ax = sns.boxplot(x=df["x_1"])

In [ ]:
plt.figure(figsize=(20,10))
c = df.corr()
sns.heatmap(c,cmap="BrBG",annot=True);

In [ ]:
df.corr().unstack().sort_values(ascending = False).drop_duplicates()

Difficult to say anything from this correlation plot. It can be spotted there there is a negative correlation with the response variable.

In [ ]:
plt.hist(y, density=True)  # density=False would make counts
plt.ylabel('Probability')
plt.xlabel('Data');

Almost a normal distribution, migth cause some troubles.

## Model Selection

There have been chosen to do a two-fold cross-validation

https://towardsdatascience.com/nested-cross-validation-hyperparameter-optimization-and-model-selection-5885d84acda

In [ ]:
#Defining the cross-validation
K = 5
CV = KFold(K, shuffle=True)

The inner loop is defined below, this is where we're doing our model selection. 
The EPE is calculated within the GridSearchCV, where RMSE is used. When calling the function .best_score_, the out is the mean of RMSE from all the folds, which is eqaclty when we want to determine which model to go with when determine the final model and the model assesment.

In [ ]:
def models(X, y, K):
    
    n_samples = X.shape[0]
    n_features = X.shape[1]
    
    InnerCV = KFold(K, shuffle=True)
    
    ##############################
    ########### MODLES ###########
    ##############################
    
    # Bagging
    model_bag = BaggingRegressor()
    
    params_bag = {'n_estimators': [20,50,100],
                  'max_samples': [0.5,1.0, n_samples//2,],
                  'max_features': [0.5,1.0, n_features//2,],
                  'bootstrap': [True, False],
                  'bootstrap_features': [True, False]}
    
    bag_grid = GridSearchCV(model_bag, params_bag, cv=InnerCV, scoring='neg_root_mean_squared_error').fit(X,y)
    best_bag = bag_grid.best_estimator_
    
    EPE_bag = bag_grid.best_score_ #this value is given as the mean
    print('Done with Bagging')
    
    # Ada Boost
    model_ada = AdaBoostRegressor()
    
    params_ada = parameters = {'n_estimators':[10, 50, 100, 500], 
                               'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1.0]}
    ada_grid = GridSearchCV(model_ada, params_ada, cv=InnerCV, scoring='neg_root_mean_squared_error').fit(X,y)
    best_ada = ada_grid.best_estimator_
    
    EPE_ada = ada_grid.best_score_ #this value is given as the mean
    print('Done with Ada Boost')
    
    # Random Forest
    model_rf = RandomForestRegressor()
    
    params_rf = {'bootstrap': [True, False],
                 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                 'max_features': ['auto', 'sqrt'],
                 'min_samples_leaf': [1, 2, 4],
                 'min_samples_split': [2, 5, 10],
                 'n_estimators': [50, 100, 200, 400, 600, 800, 1000]}
    rf_grid = GridSearchCV(model_rf, params_rf, cv=InnerCV, scoring='neg_root_mean_squared_error').fit(X,y)
    best_rf = rf_grid.best_estimator_
    
    EPE_rf = rf_grid.best_score_
    print('Done with Random Forest')
    
    # Support Vector Regression
    model_svr= svr_rbf = SVR()
    
    params_svr = parameters = {'kernel': ('linear', 'rbf','poly'), 
                              'C':[1.5, 10],
                              'gamma': [1e-7, 1e-4],
                              'epsilon':[0.1,0.2,0.5,0.3]}
    svr_grid = GridSearchCV(model_svr, params_svr, cv=InnerCV, scoring='neg_root_mean_squared_error').fit(X,y)
    best_svr = svr_grid.best_estimator_
    
    EPE_svr = svr_grid.best_score_
    print('Done with Support Vector Regression Forest')

    list_err = [EPE_bag, EPE_ada, EPE_rf, EPE_svr]
    best_models = [best_bag, best_ada, best_rf,best_svr]
    min_err = list_err.index(min(list_err))
    
    best = best_models[min_err]
    
    model = best.fit(X,y)
    
    return model

Outer fold is defined below - also where the model assesment is done

In [ ]:
EPE_test = []
EPE_train = []

for i, (train_index, test_index) in enumerate(CV.split(X,y)):
    
    print(f'Fold: {i}')
    
    X_train = X.iloc[train_index, :]
    y_train = y[train_index]
    X_test = X.iloc[test_index, :]
    y_test = y[test_index]
    
    #############################
    ### Best model - returned ###
    #############################
    
    model = models(X_train, y_train, K) #K is the number of folds
    yhat_test = model.predict(X_test)
    yhat_train = model.predict(X_train)
    
    EPE_test.append(mean_squared_error(y_test, yhat_test, squared = False))
    EPE_train.append(mean_squared_error(y_train, yhat_train, squared = False))

In [ ]:
performance_eval = pd.DataFrame([EPE_test, EPE_train], columns=['RMSE test','RMSE train'])
performance_eval.head(K)

In [ ]:
EPE = np.mean(EPE_test)
model = models(X, y, K) #final model
y_hat_new = model.predict(df_pred)

### Save in .txt files

In [ ]:
np.savetxt('predictions_s184296s163724.txt', np.array(y_hat_new), fmt='%s', delimiter=',')

In [ ]:
np.savetxt('estimatedRMSE_s184296s163724.txt', np.array(EPE_test), fmt='%s', delimiter=',')

### Final model

In [ ]:
model